# DRAFT- IBM CPD DV Connection - DRAFT


### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/db2dmc.

### First we will import a few helper classes
We need to pull in a few standard Python libraries so that we can work with REST, JSON and a library called Pandas. Pandas lets us work with DataFrames, which are a very powerful way to work with tabular data in Python. 

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas.io.json import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### The Db2 Class
Next we will create a Db2 helper class that will encapsulate the Rest API calls that we can use to directly access the Db2 Data Management Console service without having to use the user interface. 

To access the service we need to first authenticate with the service and create a reusable token that we can use for each call to the service. This ensures that we don't have to provide a userID and password each time we run a command. The token makes sure this is secure. 

Each request is constructed of several parts. First, the URL and the API identify how to connect to the service. Second the REST service request that identifies the request and the options. For example '/metrics/applications/connections/current/list'. And finally some complex requests also include a JSON payload. For example running SQL includes a JSON object that identifies the script, statement delimiters, the maximum number of rows in the results set as well as what do if a statement fails.

The full set of APIs are documents as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to that directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, api, userid, password):
        
        credentials = {'username':userid, 'password':password}
        r = requests.post(self.url+api+'/preauth/signin', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = "Bearer " + r.cookies["ibm-private-cloud-session"]
            self.headers = {'Content-Type':"application/json", 'Accept':"application/json", 'Authorization': bearerToken, 'Cache-Control': "no-cache"}
        else:
            print ('Unable to authenticate, no bearer token obtained')
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def getDataSources(self):
        return self.getRequest('/icp4data-databases/dv/zen/dvapiserver/v1/dv/datasource_nodes')
    
    def getSchemas(self):
        return self.getRequest('/icp4data-databases/dv/zen/dbapi/v4/schemas')
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/icp4data-databases/dv/zen/dbapi/v4/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/icp4data-databases/dv/zen/dbapi/v4/sql_jobs/'+jobid)
       
    def getSearchViewList(self, searchtext, show_systems="false"):
        return self.getRequest('/icp4data-databases/dv/zen/dbapi/v4/admin/schemas/obj_type/view?search_name='+searchtext+'&show_systems='+str(show_systems)+'&rows_return=200');
    
    def getSearchTableList(self, searchtext):
        return self.getRequest('/icp4data-databases/dv/zen/dbapi/v4/admin/schemas/obj_type/table?search_name='+searchtext+'&show_systems=true&rows_return=100');
               
    def postSearchObjects(self, obj_type, search_text, rows_return=100, show_systems='false', is_ascend='true'):     
        json = {"search_name":search_text,"rows_return":rows_return,"show_systems":show_systems,"obj_type":obj_type,"filters_match":"ALL","filters":[]}       
        return self.postRequest('/icp4data-databases/dv/zen/dbapi/v4/admin/'+str(obj_type)+'s',json);
            
    def getTablesInSchema(self, schema):
        return self.getRequest('/icp4data-databases/dv/zen/dbapi/v4/schemas/'+str(schema)+'/tables'); 

In [3]:
def runSQL(sqlText):

    # Run the SQL Script and return the runID for later reference 
    runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 

    # See if there are any results yet for this job
    json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

    # If the REST call returns an error return the json with the error to the calling routine
    if 'errors' in json :
        return json
    # Append the results from each statement in the script to the overall combined JSON result set
    fulljson = json

    while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
        json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

        # Get the results from each statement as they return and append the results to the full JSON 
        for results in json['results'] :
            fulljson['results'].append(results)
        # Wait 250 ms for more results
        time.sleep(0.25) 
    return fulljson

print('runSQL routine defined')

runSQL routine defined


In [4]:
def displayResults(json):

    for results in json['results']:
        print('Statement: '+str(results['index'])+': '+results['command'])
        print('Runtime ms: '+str(results['runtime_seconds']*1000))
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. Row Affected: '+str(results['rows_affected']))
        print()
print('displayResults routine defined')

displayResults routine defined


## Establishing a Connection to the Console

### Example Connections
To connect to the Db2 Data Management Console service you need to provide the URL, the service name (v4) and profile the console user name and password as well as the name of the connection profile used in the console to connect to the database you want to work with. For this lab we are assuming that the following values are used for the connection:
* Userid: db2inst1
* Password: db2inst1
* Connection: sample

**Note:** If the Db2 Data Management Console has not completed initialization, the connection below will fail. Wait for a few moments and then try it again.

In [5]:
# Connect to the Db2 Data Management Console service
Console  = 'https://zen-cpd-zen.apps.sre-dal10-demosundari-ga-02.demo.ibmcloud.com'
user     = 'admin'
password = '1qaz!QAZ'

# Set up the required connection
databaseAPI = Db2(Console)
api = '/v1'
databaseAPI.authenticate(api, user, password)
database = Console

In [6]:
r = databaseAPI.getDataSources()
print(r)
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print(', '.join(list(df)))
    display(df)
else:
    print(databaseAPI.getStatusCode(r))  

<Response [200]>
agent_class, dataSources, dscount, hostname, is_docker, node_description, node_name, os_user, port


,agent_class,dataSources,dscount,hostname,is_docker,node_description,node_name,os_user,port
0,H,None,0,dv-0,N,Not specified,AdminNode,bigsql,6414
1,H,"[{'cid': 'DB210000', 'dbname': 'BANK', 'srchos...",1,dv-0,N,Not specified,qpendpoint_3:6417,bigsql,6417
2,H,None,0,dv-0,N,Not specified,qpendpoint_4:6418,bigsql,6418
3,H,None,0,dv-0,N,Not specified,qpendpoint_2:6416,bigsql,6416
4,H,None,0,dv-0,N,Not specified,qpendpoint_5:6419,bigsql,6419
5,H,"[{'cid': 'DB210003', 'dbname': 'MORTGAGE', 'sr...",1,dv-0,N,Not specified,qpendpoint_1:6415,bigsql,6415


In [7]:
sqlText = \
'''
SELECT TABSCHEMA, TABNAME FROM SYSCAT.TABLES; 
SELECT INDSCHEMA, INDNAME FROM SYSCAT.INDEXES;
'''

displayResults(runSQL(sqlText))

Statement: 0: SELECT TABSCHEMA, TABNAME FROM SYSCAT.TABLES
Runtime ms: 128.9999932050705
    TABSCHEMA            TABNAME
0  IBMCONSOLE    TEMPLATE_CONFIG
1  IBMCONSOLE           USERCRED
2  IBMCONSOLE  USER_PROFILE_PRIV
3    QPLEXSYS            DIAGLOG
4    QPLEXSYS      ENDPOINTSTATS
5    QPLEXSYS          EXPLAINDS
6    QPLEXSYS      INSTANCE_INFO
7    QPLEXSYS          LISTNODES
8    QPLEXSYS           LISTRDBC
9    QPLEXSYS        LISTSOURCES

Statement: 1: SELECT INDSCHEMA, INDNAME FROM SYSCAT.INDEXES
Runtime ms: 2.0000000949949026
  INDSCHEMA             INDNAME
0  SYSIBM       INDNODEGROUPS02
1  SYSIBM    INDPARTITIONMAPS01
2  SYSIBM     INDPASSTHRUAUTH01
3  SYSIBM     INDPASSTHRUAUTH02
4  SYSIBM     INDPASSTHRUAUTH03
5  SYSIBM     INDPASSTHRUAUTH04
6  SYSIBM          INDPERIODS01
7  SYSIBM          INDPERIODS02
8  SYSIBM             INDPLAN01
9  SYSIBM             INDPLAN02



## Object Exploration

### List the Available Schemas in the Database

You can get the list of schemas through a REST service call. In this example the service call text was defined in the Db2 class at the start of the notebook. By default it includes both user and catalog schemas. 

If the call is successful it will return a 200 status code. The API call returns a JSON structure that we turn into a DataFrame using the normalize function. You can then list the columns of data available in the Data Frame and display the first 10 rows in the data frame. 

Many of the examples below list the columns available in the dataframe to make it easier for you to adapt the examples to your own needs. 

In [8]:
r = databaseAPI.getSchemas()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json['resources']))
    print(', '.join(list(df)))
    display(df[['name']].head(10))
else:
    print(databaseAPI.getStatusCode(r))   

definertype, name


,name
0,BIGSQL
1,CACHESYS
2,DEFAULT
3,IBMCONSOLE
4,NULLID
5,QPLEXSYS
6,SQLJ
7,SYSCAT
8,SYSFUN
9,SYSHADOOP


In [ ]:
#Run SQL Statement and Retrieve the identifier used to track the run

# Define the SQL to Run
sqlText = 'select TABSCHEMA, TABNAME from syscat.tables'

# Run SQL throught the Console SQL Editor REST Service
r = databaseAPI.runSQL(sqlText)

# Retrieve the run identifier from the SQL Editor Service
runID = databaseAPI.getJSON(r)['id'] 

# Using the saved runID retrieve a JSON description of the run
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID)) 

# Extract the status of the run from the JSON return and print the runID as well as its status
if json['results'] == [] :
    print(runID+" "+json['status']) 
else :
    print('JSON Results')
    print(json['results']) 

In [ ]:
# ONLY RUN THIS STEP IF THERE WERE NO RESULTS FROM THE PREVIOUS STEP
# Even if the status is running. You need to extract and process the results before you get more results.
# Once you have retrieved the results you cannot run the same call a second time successfully.
# The JSON structure is only available on the first call.
json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))
print(json)

In [ ]:
if json['results'] != []:
        results = json['results'][0]
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. No results')

### Next Steps
Try the [Analysing SQL Workloads](http://localhost:8888/notebooks/Db2_Data_Management_Console_SQL.ipynb). It contains extensive examples on how to run workloads that contain multiple SQL Statements across multiple databases and then measure their performance. 

Also try building some of your own reports based on the examples in this hands on lab. There are additional functions included in the Db2 class that we haven't explored yet in this lab. You can also include the Db2 class into your own notebook by including the [dmc_setup notebook](http://localhost:8888/notebooks/dmc_setup.ipynb)

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]